In [1]:
#pycharm 測試
import sys,os
sys.path.append("..")
import django
django.setup()
import quandl
from django.conf import settings
from us_data.models import *
from etl.import_sql import *
import logging
import datetime
from io import StringIO
import requests

logging.basicConfig(level=logging.INFO)

In [3]:
class SharadarCrawler:
    def __init__(self, date=None):
        self.date = date
        quandl.ApiConfig.api_key = settings.CONFIG_DATA.get("QUANDL_API_KEY")

    def download_sep(self):
        df = quandl.get_table('SHARADAR/SEP', date=self.date, paginate=True)
        if len(df) < 1:
            return None
        df = df.drop(columns='lastupdated')
        df = df.rename(columns={'ticker': 'stock_id'})
        return df

    def download_sp500(self):
        engine = create_connect_engine(SharaderSp500)
        conn = engine.connect()
        db_name=get_connect_db_name(SharaderSp500)
        sql = f"DELETE FROM {db_name}.sharader_sp500"
        conn.execute(sql)
        df = quandl.get_table('SHARADAR/SP500', paginate=True)
        if len(df) < 1:
            return None
        df = df.drop(columns='note')
        df = df.rename(columns={'ticker': 'stock_id'})
        df = df.replace({'N/A': None})
        df = df[df['action'] == 'current']
        df['id'] = [i for i in range(1, len(df) + 1)]
        return df

    def download_sf1(self):
        df = quandl.get_table('SHARADAR/SF1', calendardate=self.date, paginate=True)
        if len(df) < 1:
            return None
        df = df.rename(columns={'ticker': 'stock_id', "calendardate": 'date'})
        return df

    def download_sf3a(self):
        df = quandl.get_table('SHARADAR/SF3A', calendardate=self.date, paginate=True)
        if len(df) < 1:
            return None
        df = df.rename(columns={'ticker': 'stock_id', "calendardate": 'date'})
        return df

    def download_sf3b(self):
        df = quandl.get_table('SHARADAR/SF3B', calendardate=self.date, paginate=True)
        if len(df) < 1:
            return None
        df = df.rename(columns={'calendardate': 'date'})
        return df

    def download_actions(self):
        df = quandl.get_table('SHARADAR/actions', date=self.date, paginate=True)
        if len(df) < 1:
            return None
        df = df.rename(columns={'ticker': 'stock_id', 'calendardate': 'date'})
        df = df.replace({'N/A': None})
        return df

    def download_daily(self):
        df = quandl.get_table('SHARADAR/DAILY', date=self.date, paginate=True)
        if len(df) < 1:
            return None
        df = df.drop(columns='lastupdated')
        df = df.rename(columns={'ticker': 'stock_id'})
        return df

    def download_eventcodes(self):
        df = quandl.get_table('SHARADAR/EVENTCODES', paginate=True)
        if len(df) < 1:
            return None
        #         df=df.rename(columns={'ticker': 'stock_id','calendardate':'date'})
        #         df=df.replace({'N/A': None})
        return df

    def download_tickers(self):
        df = quandl.get_table('SHARADAR/TICKERS', paginate=True)
        if len(df) < 1:
            return None
        return df


    
# df=SharadarCrawler(['2020-6-16','2020-10-12']).download_sep() 
# df=SharadarCrawler().download_sp500()
# df=SharadarCrawler(['2020-06-30']).download_sf1()
# df=SharadarCrawler(['2018-03-31']).download_sf3b()
# df=SharadarCrawler(['2020-08-31']).download_actions()
# df=SharadarCrawler(['2020-08-31']).download_daily()
# df=SharadarCrawler(['2020-08-31']).download_tickers()
# df

In [ ]:
df.head(55)

# 日財報爬蟲

In [ ]:
#爬取2020-09-01到2020-09-10
crawler=CrawlerProcess(SharadarCrawler,'download_sep',SharadarSep,'date_range',use_date_list=True)
# print(crawler)

# # 指定日期區間爬蟲
# crawler.specified_date_crawl('2020-6-16','2020-10-12')

# 自動檢查最後日期爬取
crawler.auto_update_crawl()

#  季財報爬蟲

In [ ]:
# season generator
# def us_seasonly( start_date, end_date):
#     start_date=datetime.datetime.strptime(start_date, "%Y-%m-%d")
#     end_date=datetime.datetime.strptime(end_date, "%Y-%m-%d")
#     present_year = datetime.datetime.now()
#     calendardates = pd.date_range(str(start_date.year-1), str(end_date.year+1), freq='Q').astype(str).to_list()
#     return calendardates

# us_seasonly( '2018-09-01','2020-09-06')

In [4]:
crawler=CrawlerProcess(SharadarCrawler,'download_sf1',SharadarSf1,'us_season_range',time_sleep=0,use_date_list=False, jump_update=True)
print(crawler)
# # 指定日期區間爬蟲
# crawler.specified_date_crawl('2020-09-01','2020-09-06')

# 自動採當今日期前後爬取
crawler.auto_update_crawl()

sharadar_sf1 
table_earliest_date:1990-06-30 00:00:00
table_latest_date:2020-12-31 00:00:00


INFO:etl.import_sql:Finish!download data from 2019-09-30 to 2019-09-30
INFO:etl.import_sql:Finish!download data from 2019-12-31 to 2019-12-31
INFO:etl.import_sql:Finish!download data from 2020-03-31 to 2020-03-31
INFO:etl.import_sql:Finish!download data from 2020-06-30 to 2020-06-30
INFO:etl.import_sql:Finish!<class 'us_data.models.SharadarSf1'> date:from 2020-09-30 to 2020-09-30 bulk_create:14416
INFO:etl.import_sql:Finish!<class 'us_data.models.SharadarSf1'> date:from 2020-09-30 to 2020-09-30 bulk_update:0
INFO:etl.import_sql:Finish!download data from 2020-09-30 to 2020-09-30
INFO:etl.import_sql:Finish!<class 'us_data.models.SharadarSf1'> date:from 2020-12-31 to 2020-12-31 bulk_create:306
INFO:etl.import_sql:Finish!<class 'us_data.models.SharadarSf1'> date:from 2020-12-31 to 2020-12-31 bulk_update:0
INFO:etl.import_sql:Finish!download data from 2020-12-31 to 2020-12-31


'Finish!download data from 2019-09-30 to 2020-12-31'

# S&P 500

In [ ]:
def crawler_task_once(crawler, func, model):
    df = getattr(eval(crawler)(), func)()
    importer = SqlImporter.add_to_sql(eval(model), df)
    return importer



In [ ]:
crawler_task_once('SharadarCrawler','download_sp500','SharaderSp500')

# Nasdaq 100

In [ ]:
class SlickchartsCrawler:
    def __init__(self, date=None):
        self.market_range=['dowjones','nasdaq100','sp500']
    def index_components(self,index_name:str):
        headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) '
                          'Chrome/86.0.4240.75 Safari/537.36'}
        # dowjones,nasdaq100,sp500
        try:
            url=f"https://www.slickcharts.com/{index_name}"
        except Exception as error:
            logger.error(error)
            df = pd.DataFrame()
            return df
        r = requests.get(url, headers=headers)
        df = pd.read_html(StringIO(r.text))[0]
        df=df.rename(columns={'Company':'name','Symbol': 'stock_id','Weight':'weight'})
        df=df[['stock_id','name','weight']]
        df['index_name']=index_name
        return df
    def main(self):
        try:
            df=pd.concat([self.index_components(i) for i in self.market_range])
            df['id']=[i for i in range(1,len(df)+1)]
        except Exception as error:
            logger.error(error)
            return None
        engine = create_connect_engine(IndexComponents)
        conn = engine.connect()
        sql=f"DELETE FROM dev_us_data.index_components"
        conn.execute(sql)        
        return df
            

# SlickchartsCrawler().main()


In [ ]:
crawler_task_once('SlickchartsCrawler','main','IndexComponents')

## once import

In [ ]:
# import logging
# from etl.import_sql import SqlImporter, CrawlerProcess
# from us_data.crawlers import *
# from us_data.models import *

# def crawler_task_once(crawler, func, model):
#     df = getattr(eval(crawler)(), func)()
#     importer = SqlImporter.add_to_sql(eval(model), df)
#     return importer

# crawler_task_once('SharadarCrawler','download_sp500','SharaderSp500')

In [ ]:
for a,i in enumerate(df):
    if len(i)>20:
        print(a)
        

In [ ]:
create_connect_engine(SharaderSp500)

In [ ]:
import pandas as pd
df=pd.DataFrame({'a':[1,2,3,4],'b':[4,2,3,4]})
df

In [ ]:
df = df.apply(pd.to_numeric)
df

In [ ]:
df['a']

In [ ]:
'="0050"'.replace('=', '')